# Step 2.5: Generate Training Set

## Setup

In [ ]:
import pandas as pd
import numpy as np
import itertools
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt

#### Load Dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/lisavirginia/clinical-abbreviations/master/modules/Step2Output_Clinical_Abbreviation_Acronym_Crosswalk.csv',
                 sep='|',
                 header=0,
                 index_col=False,
                 na_filter=False,
                 dtype=object)

In [ ]:
df.sample(3, random_state=0)

## Initial Filtering

Identifies **most similar string** that is **not equivalent**.

We used the standard Levenshtein distance similarity ratio available [here](https://github.com/seatgeek/fuzzywuzzy).

#### Use Normalized Long Form Where Possible

In [ ]:
df['TrainLF'] = np.where(df['NormLF']=='', df['LF'], df['NormLF'])

In [ ]:
df.sample(3, random_state=0)

#### Extract Unique Long Forms

In [ ]:
uniq_LFs = set(df['TrainLF'].values)

#### Compute Similarities

In [ ]:
train = []

In [ ]:
for pair in itertools.combinations(uniq_LFs, 2):
    distance = fuzz.ratio(pair[0], pair[1])
    if distance > 80:
        pair_dictionary = {'LF1':pair[0], 'LF2':pair[1], 'distance':distance} 
        train.append(pair_dictionary)

In [ ]:
train = pd.DataFrame(train)  

In [ ]:
train.sample(3, random_state=0)

## Highly Similar

Assume highly similar long forms are synonymous (~1 character difference).

In [ ]:
TrainSet2 = train[train.distance >= 98]
TrainSet2 = TrainSet2.drop(columns=['distance'])
TrainSet2['Synonym'] = 'Y'

In [ ]:
TrainSet2.sample(3, random_state=0)

#### Export

In [ ]:
TrainSet2.to_csv('TrainingSet2_Annotated.csv',
                 index=False,
                 header=True,
                 sep='|')

## Random Subset

Select random subset of moderately similar long forms for manual annotation.

In [ ]:
TrainSet1 = train[train.distance < 98]
TrainSet1 = TrainSet1.sample(frac=0.10)
TrainSet1 = TrainSet1.drop(columns=['distance'])
TrainSet1['Synonym'] = ''

In [ ]:
TrainSet1.sample(3, random_state=0)

#### Export

In [ ]:
TrainSet1.to_csv('TrainingSet1.csv',
                 index=False,
                 header=True,
                 sep='|')